## Environement 

In [1]:
import datetime
import logging


import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import datetime as dt
from datetime import timedelta, datetime
import sqlalchemy
from sqlalchemy import create_engine

import json
import requests
import pandas as pd
from six import string_types
from six.moves.urllib.parse import urlencode, urlunparse  # noqa

from datetime import date
from datetime import timedelta

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
from datetime import datetime

import custome_credential
POSTGRES_ADDRESS = custome_credential.POSTGRES_ADDRESS
POSTGRES_PORT = custome_credential.POSTGRES_PORT
POSTGRES_USERNAME = custome_credential.POSTGRES_USERNAME
POSTGRES_PASSWORD = custome_credential.POSTGRES_PASSWORD
POSTGRES_DBNAME = custome_credential.POSTGRES_DBNAME

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

In [2]:
df=pd.read_csv('###############.csv')
df_save=df

## prepare Datatable 

In [3]:
# only select numerical rating and favorate flavor 
df=df[[col for col in df.columns if "rating" in col]]
df['survey_favorite_flavor']=df_save['survey_favorite_flavor']

C:\Users\MarkMao\anaconda3\envs\powerpy3.7\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Stag 1 model

1. Website filtering

2. Cosine similarity

Bosth require defining the qualitative matrix for each flavor 

## Model selection
simplest stage 2 model which take a string(favorate flavor as input)

### Weighted_rating: it is weighted average for certain favorate flavor by how much rating for each flavor. 
### It is meant to emphasize the popularity for each flavor

In [4]:

def weighted_rating(favorate):
    tmp_df=df[df['survey_favorite_flavor']==favorate]
    tmp_df=tmp_df.drop(columns=['survey_favorite_flavor'])
    #number of valid rating for each flavor
    v = tmp_df.count()
    #average rating of the each flavor
    R = df.mean()
    #mean number of valid rating across the whole table
    C= tmp_df.count().mean()
    
    # Calculation weighted score based on average rating plus constant of weighted number of valid rating
    return (R) + ((v-C)/v.std())/2

### simple_rating: it is average for certain favorate flavor . 
### It is meant to emphasize the only the average rating for each flavor

In [5]:
def simple_rating(favorate):
    tmp_df=df[df['survey_favorite_flavor']==favorate]
    tmp_df=tmp_df.drop(columns=['survey_favorite_flavor'])
    #number of valid rating for each flavor
    v = tmp_df.count()
    #average rating of the each flavor
    R = df.mean()
    #mean number of valid rating across the whole table
    C= tmp_df.count().mean()
    
    # Calculation weighted score based on average rating plus constant of weighted number of valid rating
    return (R) 

In [ ]:
#print top 5 high score
weighted_rating('BASE-PURECUC').sort_values(ascending=False).head(5)

In [ ]:
#print top 5 high score
simple_rating('BASE-LIFECP').sort_values(ascending=False).head(5)

## recommendation
It can be a good idea to test how people like those two kind of recommendation model by AB testing 

# SVD model, Collaborative filtering 

In [9]:
df_svd=df[[col for col in df.columns if "rating" in col]]

In [10]:
df_svd=df_svd.reset_index().rename(columns={'index':'user_id'})

In [11]:
keys = [col for col in df.columns if "rating" in col]
df_svd=pd.melt(df_svd, id_vars='user_id', value_vars=keys, value_name='key')

In [13]:
df_svd=df_svd[['user_id', 'variable', 'key']]

In [14]:
df_svd=df_svd.dropna(subset=['key'])

In [16]:
from surprise import Dataset
from surprise import Reader
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df_svd[['user_id', 'variable', 'key']], reader)

In [17]:
reader

In [29]:
from surprise import SVD
from surprise.model_selection import cross_validate
svd = SVD(verbose=True, n_epochs=20,n_factors =200,lr_all =0.01)
cross_validate(svd, data, measures=['RMSE', 'MAE'],return_train_measures=True, cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

{'test_rmse': array([2.28687689, 2.29204858, 2.28291841]),
 'train_rmse': array([0.78501097, 0.79218574, 0.74706859]),
 'test_mae': array([1.76850375, 1.77055755, 1.76513471]),
 'train_mae': array([0.46063897, 0.46031067, 0.4447657 ]),
 'fit_time': (87.44206285476685, 85.75314474105835, 87.09716367721558),
 'test_time': (3.4670567512512207, 3.833305835723877, 3.779313802719116)}

# grid search

In [30]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [5, 10,15], 'lr_all': [0.002, 0.005,0.001],'n_factors': [200,100]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3,joblib_verbose=1)
grid_search.fit(data)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 34.2min finished


In [31]:
best_svd = grid_search.best_estimator['rmse']
cross_validate(best_svd, data, measures=['RMSE', 'MAE'],return_train_measures=True, cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    2.3033  2.3069  2.3034  2.3045  0.0017  
MAE (testset)     1.7806  1.7839  1.7825  1.7823  0.0014  
RMSE (trainset)   0.5235  0.5265  0.5193  0.5231  0.0030  
MAE (trainset)    0.3709  0.3721  0.3708  0.3712  0.0006  
Fit time          60.74   60.29   60.67   60.57   0.20    
Test time         3.88    4.07    3.78    3.91    0.12    


{'test_rmse': array([2.30329452, 2.30686657, 2.30336345]),
 'train_rmse': array([0.52350755, 0.52654275, 0.51933306]),
 'test_mae': array([1.78055395, 1.78391642, 1.78249863]),
 'train_mae': array([0.370898  , 0.37209206, 0.37075537]),
 'fit_time': (60.74272179603577, 60.29292869567871, 60.666943311691284),
 'test_time': (3.8761606216430664, 4.073827266693115, 3.780325174331665)}

In [42]:
svd = SVD(verbose=True, n_epochs=5,n_factors =200,lr_all =0.005)
cross_validate(svd, data, measures=['RMSE', 'MAE'],return_train_measures=True, cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    2.3403  2.3468  2.3428  2.3433  0.0027  
MAE (testset)     1.8022  1.8050  1.8038  1.8037  0.0012  
RMSE (trainset)   1.5253  1.5181  1.5084  1.5172  0.0069  
MAE (trainset)    1.1403  1.1375  1.1273  1.1350  0.0056  
Fit time          21.00   21.15   21.24   21.13   0.10    
Test time         3.83    4.02    4.10    3.98    0.11    


{'test_rmse': array([2.34030708, 2.34680504, 2.34277811]),
 'train_rmse': array([1.52525082, 1.51805376, 1.50836042]),
 'test_mae': array([1.80218754, 1.80502395, 1.80382748]),
 'train_mae': array([1.14031516, 1.13750555, 1.12731834]),
 'fit_time': (20.995699405670166, 21.14903211593628, 21.237251043319702),
 'test_time': (3.8278257846832275, 4.021021366119385, 4.09672737121582)}

In [40]:
grid_search.best_params

{'rmse': {'n_epochs': 15, 'lr_all': 0.005, 'n_factors': 200}}

# optimal parameter: 
### Overfitting but lowest RMSE: 

Number of factors: 200

Number of recommendation algorithm iterations: 15

Learning rate: 0.005

### not overfitting with relatively low RMSE: 

Number of factors: 200

Number of recommendation algorithm iterations: 5

Learning rate: 0.005

In [46]:
#fit the model with all data we had
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4


In [48]:
#make a prediction 
testset = trainset.build_testset()
predictions = svd.test(testset)